---

# 🧪 Testing with Pydantic — Unit & Property Tests

### 🎯 Intent

Test **Pydantic v2 models** with happy/negative paths, parametrized inputs, and property-based fuzzing.

---

### 🧩 Core Components

1. **✅ Happy Path**

   ```python
   def test_user_ok():
       u = User(id="1", name="Mukesh")
       assert u.id == 1
       assert u.model_dump()["name"] == "Mukesh"
   ```

2. **❗ Negative Cases**

   ```python
   import pytest
   from pydantic import ValidationError

   def test_user_invalid():
       with pytest.raises(ValidationError) as e:
           User(id="x", name="")
       errs = e.value.errors()
       assert any(er["loc"] == ("id",) for er in errs)
   ```

3. **📦 Parametrized Tests**

   ```python
   @pytest.mark.parametrize("payload,ok", [
       ({"id":1,"name":"A"}, True),
       ({"id":"2","name":"B"}, True),
       ({"id":"x","name":"C"}, False),
   ])
   def test_user_matrix(payload, ok):
       if ok: assert User.model_validate(payload)
       else: pytest.raises(ValidationError, User.model_validate, payload)
   ```

4. **🧬 Property-Based (Hypothesis)**

   ```python
   from hypothesis import given, strategies as st

   @given(st.integers(), st.text(min_size=1, max_size=20))
   def test_user_props(i, s):
       u = User(id=i, name=s)
       assert isinstance(u.id, int)
   ```

5. **📜 Schema Stability**

   ```python
   def test_schema(snapshot):
       schema = User.model_json_schema(mode="serialization")
       snapshot.assert_match(schema)
   ```

6. **🔒 Secrets & Masking**

   ```python
   def test_password_hidden():
       out = Account(password="secret").model_dump()
       assert out["password"] == "****"
   ```

7. **📥 Bulk Validation**

   ```python
   from pydantic import TypeAdapter

   def test_bulk_errors():
       ta = TypeAdapter(list[User])
       with pytest.raises(ValidationError) as e:
           ta.validate_python([{"id":1,"name":"ok"}, {"id":"x","name":"bad"}])
       assert any("id" in str(er["loc"]) for er in e.value.errors())
   ```

8. **⚙️ Strict & Extra Tests**

   ```python
   def test_strict():
       class StrictUser(User): model_config = {"strict": True}
       with pytest.raises(ValidationError):
           StrictUser(id="1", name="A")
   ```

9. **🛠 Fixtures/Factories**

   ```python
   def user_payload(**over):
       base = {"id":1,"name":"Mukesh"}
       base.update(over); return base
   ```

10. **🌐 FastAPI Smoke**

```python
from fastapi.testclient import TestClient
def test_signup(client: TestClient):
    r = client.post("/signup", json={"name":"", "email":"bad"})
    assert r.status_code == 422
```

---
